In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn.functional as F
from BcomMEG import *


In [ ]:
# So one option would be to use ResNet as the pretrained and try to fine-tune it. 
# I'm not opposed to this, but it would require messing with the first layer, and
# making sort of arbitrary downscaling of the MEG data. We would have to be clever
# about this. I think I'm just going to try to train my own for right now and see
# how it goes
# resnet = models.resnet50(weights='ResNet50_Weights.DEFAULT')
# print(resnet.conv1)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/ciprianbangu/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
  0%|          | 0.00/97.8M [00:00<?, ?B/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
100%|██████████| 97.8M/97.8M [00:03<00:00, 32.6MB/s]


Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [ ]:
dir = '/Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT'
subjects = ['BCOM_18_2', 'BCOM_18_3', 'BCOM_18_4']
data = BcomMEG(dir=dir, subjects=subjects, picks=picks)

#TODO: There needs to be a default mapping because otherwise if you initialize a dataset that is missing some of the
        # labels, the indexes will get overun. This shouldn't be a problem if you load all the data,
        # but if you don't then you could have major isses. 
        # I think it might just have to be manually initialized in the class attributes?
        
        
# get all the syllables in the dataset accross included subjects        
syllables = [] 
for subject in data:
    syllables.extend(data[subject].keys()) #put em into a list
syllables = set(syllables) #get uniques
        

# Map syllables to categorical vars (ints) and ints to their syllable counterparts
label_to_int_map = {label: index for index, label in enumerate(syllables)}
int_to_label_map = {index: label for index, label in enumerate(syllables)}

#Extract the labels again this should probably be a method in the data_dict class
label = []
for subject in data:
        label.extend(label_to_int_map[syllable] for syllable in data[subject])
label


At least for now, we are gunna be trying all-class covert. But it would be interesting to try binary Covert/Overt, or smaller classes - like breaking them down based on the vowel

In [ ]:
class MEGDataset_Conv(Dataset):
    def __init__(self, data_dict, label_map):
        # Ok, now the project of doing this correctly starts
        self.data_dict = data_dict 
        self.label_map = label_map #actually, I think this should be a property of this class, because it only really matters for the decoding. 

        self.labels = self.get_labels(self.data_dict, self.label_map)

        self.data, _  = data_to_tensor(data_dict) #NB! this is already a torch tensor, and also returns indexes
                                                  # which we want to avoid (the '_')


    def get_labels(self, data_dict, label_map):
        labels = []
        for subject in data_dict:
            labels.extend(label_map[syllable] for syllable in data_dict[subject])
        return labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        sample = self.data[index]
        label = self.labels[index]

        # if self.transform:
        #     sample = self.transform(sample)

        return sample, label
    


In [116]:
dataset = MEGDataset_Conv(data, label_to_int_map)